In [1]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import operator
import spacy
nlp = spacy.load('en_core_web_lg', disable=["ner","tagger","parser"])

from gensim.models import FastText, KeyedVectors

In [2]:
%run dataPreparation.ipynb

In [3]:
google_model = KeyedVectors.load_word2vec_format('~/Documents/GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000) 
wikipedia_model = KeyedVectors.load_word2vec_format('~/Documents/wiki_model.bin', binary=True, limit=500000) 

In [4]:
def getMostSimilarTag(user_input):
    answer = {tag:0 for tag in clean_tags.keys()}
    for noun in user_input: #firstly choose the tag categorie
        try:
            google_res = {tag:google_model.similarity(noun, lemmatizer.lemmatize(tag)) for tag in clean_tags.keys()}
            google_answer = max(google_res.items(), key=operator.itemgetter(1))
            answer[google_answer[0]] += 1
        except KeyError:
            print("No embedding in google model")

        doc = nlp(noun)
        doc2 = nlp(" ".join([tag for tag in clean_tags.keys()]))
        spacy_res = {i.text:i.similarity(doc[0]) for i in doc2}
        spacy_answer = max(spacy_res.items(), key=operator.itemgetter(1))
        answer[spacy_answer[0]] += 1
        
        try:
            wiki_res = {tag:wikipedia_model.similarity(lemmatizer.lemmatize(noun), lemmatizer.lemmatize(tag)) for tag in clean_tags.keys()}
            wiki_answer = max(wiki_res.items(), key=operator.itemgetter(1))
            answer[wiki_answer[0]] += 1
        except KeyError:
            print("No embedding in wiki model")

        
        
        
    #print(answer)
        

    most_likely_tag = max(answer.items(), key=operator.itemgetter(1))[0] #than concreete tag type
    doc = nlp(most_likely_tag)
    final_res = dict()
    for word in clean_tags[most_likely_tag]:
        doc2 = nlp(word)
        final_res.update({word:doc2[0].similarity(doc[0])})
    #print(final_res)
    
    return final_res


In [5]:
getMostSimilarTag(["meet friends"])

No embedding in google model
No embedding in wiki model
{'food': 1, 'shopping': 0, 'nightlife': 0, 'museums': 0, 'sightseeings': 0, 'nature': 0}
{'italian': 0.399834, 'coffee': 0.5719115, 'french': 0.39063978, 'sushi': 0.56590277, 'indian': 0.35212925, 'meat': 0.679823, 'seafood': 0.6674726, 'pizza': 0.59247416, 'international': 0.3518566, 'vegetarian': 0.6237852, 'fast': 0.3947091, 'thai': 0.35116133, 'mexican': 0.43580604, 'restaurant': 0.63396895, 'asian': 0.3179651, 'chinese': 0.4226744, 'diner': 0.4864095, 'american': 0.3843167, 'steakhouse': 0.32842082}


/usr/lib/python3.7/runpy.py:193: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  "__main__", mod_spec)


('food', 0.34991303)